In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import datetime
import random
from selenium import webdriver
import time


In [13]:
base_url = 'https://www.worldweatheronline.com/ikorodu-weather-history/lagos/ng.aspx'
date = datetime.datetime(2024, 8, 31)
weather_df = pd.DataFrame()

In [16]:
def get_next_date(date=date, days = 1) :
    offset = datetime.timedelta(days=days)
    return date - offset

In [26]:
driver = webdriver.Chrome()
driver.get(base_url)

In [8]:
def get_data():
    html = driver.page_source
    df_list = pd.read_html(StringIO(html))

    df =  df_list[0]
    df.columns = df.iloc[0]

    time = df['Time'].str[:2]
    temperature = df['Time'].str[5:]

    df['Time'] = time
    df['Temperature'] = temperature

    df = df[['Time', 'Temperature', 'Forecast', 'Rain', 'Rain %', 'Cloud', 'Pressure', 'Wind', 'Gust']][2:].reset_index(drop=True)

    soup = BeautifulSoup(html)
    table = soup.select('table.days-details-table')[0]

    arrows = table.select('svg.days-arrow-white')
    
    directions = [r.attrs['style'] for r in arrows]
    df['Direction (deg)'] = directions
    df.rename(columns={'Time': 'Time (hr)', 'Temperature': 'Temperature (°c)', 'Forecast': 'Forecast (°c)', 'Rain': 'Rain (mm)', 'Rain %': 'Rain (%)', 'Cloud': 'Cloud (%)', 'Pressure': 'Pressure (mb)', 'Wind': 'Wind (km/h)', 'Gust': 'Gust (km/h)'}, inplace=True)

    day_information_div = soup.select('div.days-inform')[0]
    info = day_information_div.select('span')
    extra_information = []

    for i in info:
        time_text = i.text.replace('AM', '').replace('PM', '')
        # time_text = 
        # hours, minutes = map(int, time_text.split(":"))
        # hours = hours if i.text.count('AM') else hours + 12
        # minutes = (hours * 60) + minutes

        extra_information.append(time_text)

    df['Moonrise (hr)'] = extra_information[0]
    df['Moonset (hr)'] = extra_information[1]
    df['Sunrise (hr)'] = extra_information[2]
    df['Sunset (hr)'] = extra_information[3]


    # df['Time (hr)'] = df['Time (hr)'].astype(int)
    # df['Temperature (°c)'] = df['Temperature (°c)'].str.replace(' °c', '').astype(int)
    # df['Forecast (°c)'] = df['Forecast (°c)'].str.replace(' °c', '').astype(int)
    # df['Rain (mm)'] = df['Rain (mm)'].str.replace(' mm', '').astype(float)
    # df['Rain (%)'] = df['Rain (%)'].str.replace('%', '').astype(float)
    # df['Cloud (%)'] = df['Cloud (%)'].str.replace('%', '').astype(float)
    # df['Pressure (mb)'] = df['Pressure (mb)'].str.replace(' mb', '').astype(float)
    # df['Wind (km/h)'] = df['Wind (km/h)'].str.replace(' km/h', '').astype(float)
    # df['Gust (km/h)'] = df['Gust (km/h)'].str.replace(' km/h', '').astype(float)

    return df

In [7]:
def search(date = get_next_date().strftime("%d-%m-%Y")) :
    timeout = random.randint(5, 10)    
    dateInput = driver.find_element('xpath', '//input[@type="date"]')
    dateInput.send_keys(date)

    button = driver.find_element('xpath', '//input[@type="submit"]')
    button.click()

    time.sleep(timeout)
    df = get_data()
    df['Date'] = date

    return df

In [ ]:
flag = True

while flag:
    current_date = date.strftime("%d-%m-%Y")
    try:
        df = search(current_date)
    except ValueError:
        time.sleep(30)
    except TimeoutError:
        time.sleep(30)

    weather_df = pd.concat([weather_df, df], ignore_index=True)
    date = get_next_date(date)
    l = len(weather_df)
    flag = l/8 != 1000
    print(l)

weather_df


In [29]:
weather_df.to_csv('weather.csv')